In [1]:
import QUANTAXIS as QA
import pandas as pd
import numpy as np
from database import DB_CONN
from pymongo import ASCENDING, UpdateOne

In [2]:
start = '2015-01-01'
end = '2018-09-30'

index_code_list = ['000001', '000300', '399001', '399005', '399006']
_df = pd.read_excel('data/stock_basic.xlsx', header=0, dtype={'code':str})
stock_code_list = _df.code.tolist()

In [51]:
# 替换成stock code就好
df = QA.QA_fetch_index_day_adv(index_code_list, start, end).data
df_nonind = df.reset_index()
df_nonind.date = df_nonind.date.apply(lambda x : x.strftime('%Y-%m-%d'))

In [32]:
daily = DB_CONN['daily']
daily_hfq = DB_CONN['daily_hfq']
daily.create_index(
    [('code',ASCENDING), ('date',ASCENDING), ('index',ASCENDING)])
daily_hfq.create_index(
    [('code',ASCENDING), ('date',ASCENDING), ('index',ASCENDING)])

'code_1_date_1_index_1'

In [58]:
import time

In [60]:
tic = time.process_time()

update_requests = []
for i in df_nonind.index:
    doc = dict(df_nonind.loc[i, ['code', 'date', 'open', 'high', 'low', 'close','volume',
                      ]])
    doc.update({'index':True})
    update_requests.append(
        UpdateOne(
        {'code':doc['code'], 'date':doc['date']},
        {'$set':doc},
        upsert=True)
    )
if len(update_requests) > 0:
    update_result = daily.bulk_write(update_requests, ordered=False)
    print('保存日线数据,插入：%4d条, 更新：%4d条' %
      (update_result.upserted_count, update_result.modified_count),
      flush=True)

toc = time.process_time()
print('cost time : %.2fs' % (toc-tic))

保存日线数据,插入：4575条, 更新：   0条
cost time : 4.88s


In [46]:
# df_nonind = df_nonind.assign(index=True)

In [55]:
# df_nonind.loc[0]

In [56]:
# update_result = [dict(df_nonind.loc[i, ['code', 'date', 'open', 'high', 'low', 'close','volume',
#                       'index']]) for i in df_nonind.index]


In [57]:
# from pymongo import UpdateMany